# BePro Get Events

Accesses the BePro API, gets the 1 currently available league, season and match. Then uses an event function to extract an event dataframe from the matches event data

In [4]:
import json
import sys
sys.path.append("..")
import math
import re
from flatten_json import flatten
import pandas as pd
import UtilFunctions.util_functions as util_functions
ACCESS_TOKEN = 'token'
import requests
headers = {
    'AUTHORIZATION': 'Bearer token',
}
#Replace token with API token supplied by BePro

### Get Leagues, Season & Match ID
Uses API function to get JSON on League and match data

In [ ]:
#Change API functions to the correct API calls as supplied by BePro

In [ ]:
"""#This function assumes use of only one league, if there are more, alter it so it gets the league you are looking for by changing league_json line
def get_league(ACCESS_TOKEN):
    league = !curl -X GET api -H "AUTHORIZATION: Bearer {ACCESS_TOKEN}"
    league_json = json.loads(league[-1]).get('result')[0]
    league_id = league_json.get('id')
    return league_id, league_json

#Also assumes one season, can alter to get all seasons by removing the '[0]' on function line 2
def get_season(ACCESS_TOKEN, league_id):
    season = !curl -X GET api -H "AUTHORIZATION: Bearer {ACCESS_TOKEN}"
    season_json = json.loads(season[-1]).get('result')[0]
    season_id = season_json.get('id')
    return season_id, season_json

#Get all matches
def get_matches(ACCESS_TOKEN, season_id):
    match = !curl -X GET api -H "AUTHORIZATION: Bearer {ACCESS_TOKEN}"
    print(match)
    match_json = json.loads(re.sub(r'^.*?{', '{', match[-3]+match[-1])).get('result')
    match_ids = [m.get('id') for m in match_json]
    return match_ids, match_json

#Get single match, based on [x] on line 2
def get_match(ACCESS_TOKEN, season_id):
    match = !curl -X GET api -H "AUTHORIZATION: Bearer {ACCESS_TOKEN}"
    match_json = json.loads(match[-1]).get('result')[0]
    match_id = match_json.get('id')
    return match_id, match_json
"""

In [ ]:
def get_league():
    response= requests.get('api', headers=headers)
    league_json = response.json().get('result')[0]
    league_id = league_json.get('id')
    return league_id, league_json

def get_season(league_id):
    params = {
        'league': league_id,
    }
    response = requests.get('api', params=params, headers=headers)
    season = response.json().get('result')
    season_id = season[0].get('id')
    return season_id, season

def get_matches(season_id):
    params = {
        'season': season_id,
    }
    response = requests.get('api', params=params, headers=headers)
    matches = response.json().get('result')
    matches_id = [m.get('id') for m in matches]
    return matches_id, matches
    
def get_team_matches(team_id, season_id):
    params = {
        'season': season_id,
        'team': team_id
    }
    response = requests.get('api', params=params, headers=headers)
    team_games = response.json().get('result')
    team_game_ids = [i.get('id') for i in team_games]
    return team_game_ids, team_games

def get_lineups(match_id):
    response = requests.get('api'+str(match_id)+'/lineup', headers=headers)
    lineup = response.json().get('result')
    lineup_df = [flatten(l) for l in lineup]
    lineup_df = pd.DataFrame.from_records(lineup_df)
    #team_game_ids = [i.get('id') for i in team_games]
    return lineup_df

### Get IDs for league, season and match for 1 game

In [ ]:
suwon_team_id = 4220
league_id, league_json = get_league()
season_id, season_json = get_season(league_id)
matches_id, matches_json = get_matches(season_id)
suwon_matches_id, suwon_matches_json = get_team_matches(suwon_team_id, season_id)

### Get team lineups

In [ ]:
"""game_lineups = []
for s in suwon_matches_id:
    lineup = [flatten(l) for l in get_lineups(s)]
    lineup_df = pd.DataFrame.from_records(lineup)
    game_lineups.append(lineup_df)"""

In [ ]:
"""for i in [2,15,17,25]:
    game_lineups.pop(i)
game_lineups.reverse()"""

In [ ]:
"""for i in range(1, 35):
    home_df = pd.read_csv('data/Suwon_FC/tracking/game'+str(i)+'/home_players.csv')
    home_df = home_df.merge(game_lineups[i-1][['player_id','is_starting_lineup','position_x','position_y','position_name']],on=['player_id'],how='left')
    away_df = pd.read_csv('data/Suwon_FC/tracking/game'+str(i)+'/away_players.csv')
    away_df = away_df.merge(game_lineups[i-1][['player_id','is_starting_lineup','position_x','position_y','position_name']],on=['player_id'],how='left')
    home_df.to_csv('data/Suwon_FC/tracking/game'+str(i)+'/home_players.csv', index=False)
    away_df.to_csv('data/Suwon_FC/tracking/game'+str(i)+'/away_players.csv',index=False)"""

In [ ]:
"""lineup = [flatten(l) for l in get_lineups(26373)]
lineup_df = pd.DataFrame.from_records(lineup)[['player_id','is_starting_lineup','position_x','position_y','position_name']]
lineup_df
home_df = pd.read_csv('data/Suwon_FC/tracking/game'+str(20)+'/home_players.csv')
home_df = home_df.merge(lineup_df,on='player_id')
home_df.to_csv('data/Suwon_FC/tracking/game'+str(20)+'/home_players.csv', index=False)
away_df = pd.read_csv('data/Suwon_FC/tracking/game'+str(20)+'/away_players.csv')
away_df = away_df.merge(lineup_df,on='player_id')
away_df.to_csv('data/Suwon_FC/tracking/game'+str(20)+'/away_players.csv', index=False)"""

### Get Events for a Match ID
Formatting of API output will need to be considered and modified for this function as required when seen

In [ ]:
def get_events(match_id):
    events_df=pd.DataFrame()
    response = requests.get('api'+str(match_id)+'/event_data', headers=headers)
    num_events = response.json().get('count')
    for i in range(0, num_events,100):
        params = {'offset': i}
        response = requests.get('api'+str(match_id)+'/event_data', headers=headers, params=params)
        event_json = response.json().get('result')
        event_j = [flatten(e) for e in event_json]
        event_df = pd.DataFrame.from_records(event_j)
        events_df = pd.concat([events_df,event_df],ignore_index=True)
    return events_df

### Get Formation Changes

In [ ]:
def get_formation_changes(match_id):
    response = requests.get('api'+str(match_id)+'/formation', headers=headers)
    formations = response.json().get('result')
    formation_df = pd.DataFrame.from_records([flatten(f) for f in formations])
    return formation_df

In [ ]:
def get_formation_events_df(match_id):
    ev = pd.DataFrame(get_events(match_id)['event_time'])
    lu = get_lineups(match_id)
    team_ids = lu['team_id'].unique()
    team1_players = lu[lu['team_id'] == team_ids[0]]['player_id'].unique()
    team2_players = lu[lu['team_id'] == team_ids[1]]['player_id'].unique()
    gfc = get_formation_changes(match_id)
    team1_gfc = gfc[gfc['team_id'] == team_ids[0]].reset_index(drop=True)
    team2_gfc = gfc[gfc['team_id'] == team_ids[1]].reset_index(drop=True)
    
    for t in team1_players:
        ev[t] = 'NA'
        
    for t in team2_players:
        ev[t] = 'NA'
    
    for i,row in team1_gfc.iterrows():
        changed_time = row['changed_time']
        try:
            second_changed_time = team1_gfc.loc[i+1]['changed_time']
        except:
            second_changed_time = 1000000000
        for p in range(0,11):
            p_id = row['formation_'+str(p)+'_player_id']
            if not (math.isnan(p_id)):
                p_pos = util_functions.coords_to_pos(row['formation_'+str(p)+'_position_x'],row['formation_'+str(p)+'_position_y'])
                ev.loc[(ev['event_time'] > changed_time) & (ev['event_time'] <= second_changed_time), p_id] = p_pos
    for i,row in team2_gfc.iterrows():
        changed_time = row['changed_time']
        try:
            second_changed_time = team2_gfc.loc[i+1]['changed_time']
        except:
            second_changed_time = 1000000000
        for p in range(0,11):
            p_id = row['formation_'+str(p)+'_player_id']
            if not (math.isnan(p_id)):
                p_pos = util_functions.coords_to_pos(row['formation_'+str(p)+'_position_x'],row['formation_'+str(p)+'_position_y'])
                ev.loc[(ev['event_time'] > changed_time) & (ev['event_time'] <= second_changed_time), p_id] = p_pos
            
            
    return ev

get_formation_events_df(suwon_matches_id[3])

In [ ]:
def get_formation_from_events_df(i):
    ev = pd.read_csv('data/Suwon_FC/events/game'+str(i)+'/events_df.csv')
    match_id = ev['match_id'][0]
    ev = pd.DataFrame(ev['event_time'])
    lu = get_lineups(match_id)
    team_ids = lu['team_id'].unique()
    team1_players = lu[lu['team_id'] == team_ids[0]]['player_id'].unique()
    team2_players = lu[lu['team_id'] == team_ids[1]]['player_id'].unique()
    gfc = get_formation_changes(match_id)
    team1_gfc = gfc[gfc['team_id'] == team_ids[0]].reset_index(drop=True)
    team2_gfc = gfc[gfc['team_id'] == team_ids[1]].reset_index(drop=True)
    
    for t in team1_players:
        ev[t] = 'NA'
        
    for t in team2_players:
        ev[t] = 'NA'
    
    for i,row in team1_gfc.iterrows():
        changed_time = row['changed_time']
        try:
            second_changed_time = team1_gfc.loc[i+1]['changed_time']
        except:
            second_changed_time = 1000000000
        for p in range(0,11):
            p_id = row['formation_'+str(p)+'_player_id']
            if not (math.isnan(p_id)):
                p_pos = util_functions.coords_to_pos(row['formation_'+str(p)+'_position_x'],row['formation_'+str(p)+'_position_y'])
                ev.loc[(ev['event_time'] > changed_time) & (ev['event_time'] <= second_changed_time), p_id] = p_pos
    for i,row in team2_gfc.iterrows():
        changed_time = row['changed_time']
        try:
            second_changed_time = team2_gfc.loc[i+1]['changed_time']
        except:
            second_changed_time = 1000000000
        for p in range(0,11):
            p_id = row['formation_'+str(p)+'_player_id']
            if not (math.isnan(p_id)):
                p_pos = util_functions.coords_to_pos(row['formation_'+str(p)+'_position_x'],row['formation_'+str(p)+'_position_y'])
                ev.loc[(ev['event_time'] > changed_time) & (ev['event_time'] <= second_changed_time), p_id] = p_pos
            
            
    return ev

get_formation_from_events_df(1)

In [ ]:
#for i in range(1,35):
#    fdf = get_formation_from_events_df(i)
#    fdf.to_csv('data/Suwon_FC/formations/game'+str(i)+'/formation.csv',index=False)

In [ ]:
#for mid in suwon_matches_id:
#    fdf = get_formation_events_df(mid)
#    evdf = get_events(mid)
    #print(fdf.shape)
    #print(evdf.shape)
    #fdf.to_csv('data/Suwon_FC/formations/'+str(mid)+'.csv',index=False)

### Loop through matches list and get events dataframes for Suwon FC

In [ ]:
"""def get_suwon_match_events(suwon_id, match_ids):
    count=1
    for m in match_ids:
        events_df = get_events(m)
        events_df = util_functions.convert_bepro_to_EPTS(events_df)
        events_df.to_csv('data/Suwon_FC/events/game'+str(count)+'/events_df.csv', index=False)
        count+=1
get_suwon_match_events(suwon_team_id, suwon_matches_id)"""

### Get matches dataframe for Suwon in the right format
Need to drop 4 rows and reverse order to match tracking and chronological order

In [ ]:
"""game_df = []
for i in suwon_matches_json:
    game_details = [i.get('id'),i.get('start_time'),i.get('home_team').get('id'), i.get('away_team').get('id'), i.get('detail_match_result').get('home_team_score'), i.get('detail_match_result').get('away_team_score')]
    game_df.append(game_details)
game_df = pd.DataFrame(game_df, columns=['id','date','home_id','away_id','home_score','away_score'])
game_df = game_df.drop([2,15,17,25]).reset_index(drop=True)
game_df = game_df.reindex(index=game_df.index[::-1]).reset_index(drop=True)
#game_df.to_csv('Suwon_games.csv',index=False)"""

### Change Suwon Data so that coordinates are flipped properly

In [ ]:
#import pandas as pd

In [ ]:
#for i in range(1,35):
#    events_df = pd.read_csv('data/Suwon_FC/events/game'+str(i)+'/events_df.csv')
#    events_df['y'] = 68- events_df['y']
#    events_df.to_csv('data/Suwon_FC/events/game'+str(i)+'/events_df.csv',index=False)